# Usage: phases

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lisphilar/covid19-sir/blob/master/example/usage_phases.ipynb)

Here, how to set phases in scenario analysis will be explained. As an example, use the records in India.

## Preperation

In [ ]:
# !pip install covsirphy --upgrade
import covsirphy as cs
cs.__version__

In [ ]:
# Load datasets
data_loader = cs.DataLoader("../input")
jhu_data = data_loader.jhu()
population_data = data_loader.population()

In [ ]:
# Create instance of `Scenario` class
snl = cs.Scenario(country="India", province=None)
snl.register(jhu_data, population_data)

In [ ]:
# Summarize main scenario
snl.summary()

Note:  
This is empty because no phases are registered. Term "phase" means a sequential dates in which the parameters of SIR-derived models are fixed. The details will be explained later.

Just for demonstration, we will fix the first/end date of the records here. This is un-necessary for your analysis when you want to use all records.
Today means the reference date to determine whether a phase is a past phase or a future phase.

In [ ]:
# Fix the first/last date of the records and today
# today must be in the range of (first_date, last_date).
# Default: None means automatically specified (min/max will be used)
snl.timepoints(first_date="01Mar2020", last_date="16Jul2020", today="16Jul2020")
# Check dates
print(snl.first_date)
print(snl.last_date)
print(snl.today)
# Summarize (Empty because no phases are registered)
snl.summary()

## Set phases with S-R trend analysis
How can we set phases?

Here, we will use "S-R trend analysis" explained in [S-R trend analysis section in "Kaggle: COVID-19 data with SIR model"](https://www.kaggle.com/lisphilar/covid-19-data-with-sir-model#S-R-trend-analysis). $S$ (Susceptible) and $R$ (Recovered) show the following relationship in SIR-derived models. $S_{(R)}$ means $S$ as a function of $R$.

$$S_{(R)} = N e^{-a R}$$

where $N$ is total population and $a=\cfrac{\beta}{N \gamma}$ is a constant.  
This leads to
$$\log S_{(R)} = - a R + \log N$$

$\log S$ decreases constantly with increase of $R$, when the records follow a SIR-derived model and the parameter values of the model are constant. With logarithmic y-axis scale, plot of $(x, y) = (R, S)$ shows a line. 

The slope of the line may change when the parameter values ($\beta$ and $\gamma$) are changed. To detect the dates when the parameter values were changed, we will find the change points of $\log S$ as a function of $R$, using [ruptures: change point detection in Python](https://github.com/deepcharles/ruptures). We call the period from a change point to the next change point as a "phase" in this analysis.

In [ ]:
# Set phase with S-R trend analysis
snl.trend()
snl.summary()
# If necessary, initial (0th) phase can be disabled
# scenario.disable(phases=["0th"])

## Remove un-necessary change points manually
The slope of 4th phase seems to equal to that of 3rd phase. Here, we will combine 3rd phase and 4th phase here and reset phase names.

In [ ]:
# Before: 3rd is 12Jun2020-19Jun2020, 4th is 20Jun2020-29Jun2020
# After: 3rd is 12Jun2020-29Jun2020, 4th is the same as old 5th
snl.combine(phases=["3rd", "4th"])
snl.summary()

Show the trend with manually set change points.

In [ ]:
snl.trend(force=False)

## Separate phases manually
Because point of $(x, y) = (S, R)$ jumped on 29May2020, we will separete the 1st phase with this change point and reset phase names.

In [ ]:
# Before: 1st is 20May2020-02Jun2020
# After: 1st is 20May2020-28May2020, 2nd is 29May2020-02Jun2020
snl.separate(date="29May2020", phase="1st")
snl.summary()

In [ ]:
snl.trend(force=False)

## Disable/enable phases manually
Phases can be disable for parameter eatimation. Disabled phases can be enabled with `Scenario.enable()` method.

In [ ]:
# Disable 0th phase
snl.disable(phases=["0th"])
snl.summary()

In [ ]:
snl.trend(force=False)

In [ ]:
# Enable 0th phase
snl.enable(phases=["0th"])
snl.summary()

## Add phases one by one
In the previous sections, we started with S-R trend analysis. We can create phases without this analysis when you have a new idea to find change points.

In [ ]:
# Clear all phases
snl.clear(include_past=True).summary()

In [ ]:
# Set 0th phase to set the start date of 1st phase
snl.add(end_date="19May2020")
# Add 1st phase and disable 0th phase
snl.add(end_date="28May2020").disable(phases=["0th"])
# Add 2nd phase
snl.add(end_date="02Jun2020")
# Add 3rd phase
snl.add(end_date="11Jun2020")
# Add 4th phase
snl.add(end_date="29Jun2020")
# Add 5th phase to the last of the records
snl.add().summary()